In [11]:
# There is a bug I cannot explain at the moment,
# which you can see by uncommenting the triple-quoted comments.

import pywikibot

In [152]:
# Store things as pairs: (stack, dict) where stack is
# the stack associated to a dfs, and dict is
# a dictionary sending each neighbor of a previous element of stack
# to its parent that showed up earliest in the list.
# A possible future update is to also include the distance
# from the origin in the dict, so the image of the dict
# would be an ordered pair (parent, distance). 

"""
page3 = PageNode(pywikibot.Page(site,"Module talk:Authority control/testcases"))
"""

'\npage3 = PageNode(pywikibot.Page(site,"Module talk:Authority control/testcases"))\n'

In [180]:
# Updates a pair and returns 1 if it finds a path,
# 0 if it does not find a path but updates successfully,
# and -1 if there is nowhere to expand.
# By default, the first pair is the one updated.
# careful_bookkeeping means that the pairs will be updated correctly
# even if a path is found.
def expand_a_pair(pair1, pair2, expand_first = True, pair1_to_pair2 = True, careful_bookkeeping=True):
    if not expand_first:
        return expand_a_pair(pair2, pair1, pair1_to_pair2 = not pair1_to_pair2, careful_bookkeeping = careful_bookkeeping)

    try:
        item_to_expand, path = pair1[0].pop()

        if pair1_to_pair2:
            potential_expansions = item_to_expand.get_successors()
        else:
            potential_expansions = item_to_expand.get_predecessors()
        # good potential expansions are those that have not been seen before
        # in either dictionary.  It is not completely obvious, but not hard to show
        # that if we get stuck by avoiding words in this way, then there is no path
        # from start to end.
        good_potential_expansions = []
        for item in potential_expansions:
            """
            if str(item) == str(page3):
                the_item = item.page
                the_page3 = page3.page
                print(["LOOK HERE!"]*1000)
                print("item:",item)
                print("title:",the_item.title())
                print("site:",the_item.site)
                print("namespace:",the_item.namespace())
                print("hash:",hash(item))
                print("page3:",page3)
                print("title:",the_page3.title())
                print("site:",the_page3.site)
                print("namespace:",the_page3.namespace())
                print("hash:",hash(page3))
            
            # On cmd, printing page.namespace() gives you:
            # Namespace(id=0, custom_name='', canonical_name='', aliases=[], case='first-letter', content=True, nonincludable=False, subpages=False)
            # But I don't see how to get that here...
            """
            
            if not careful_bookkeeping:
                # if we have seen this neighbor when extending pair2
                if item in pair2[1].keys():        
                    pair1[0].append((item, path + [item]))
                    return 1
                elif item not in pair1[1].keys():
                    good_potential_expansions.append((item, path + [item]))
                    pair1[1][item] = item_to_expand
            else:
                pass #TODO

        # The dictionary has already been updated during the loop
        # so we only have to update the stack now
        pair1[0].extend(good_potential_expansions)
        
    except:
        return -1


In [178]:
# use_first means that the path works by
# travelling down the stack of pair1
# followed by going backwards down pair2;
# not use_first means the opposite.
def generate_path(pair1,pair2,use_first):
    if use_first:
        endpoint, path = pair1[0].pop()
        dict = pair2[1]
        while endpoint != None:
            endpoint = dict[endpoint]
            path.append(endpoint)
        path = path[:-1]
    else:
        endpoint, path = pair2[0].pop()
        dict = pair1[1]
        while endpoint != None:
            endpoint = dict[endpoint]
            path.append(endpoint)
        path = path[:-1]
        path.reverse()
    return path

In [161]:
def bidirectional_DFS(start, end):
    stack1 = [(start,[start])]
    stack2 = [(end,[end])]
    dict1 = {start:None}
    dict2 = {end:None}

    use_first = True
    while True:
        """
        print(page3 in dict1.keys())
        print(hash(page3) in [hash(word) for word in dict1.keys()])
        print(str(page3) in [str(word) for word in dict1.keys()])
        print(page3 in stack1[-1][1])
        print(str(page3) in [str(word) for word in stack1[-1][1]])
        print("STACKS")
        for word in stack1[-1][1]:
            print(word)
        #print("DICTS")
        #for word in dict1.keys():
            #print(word)
        """
            
        result = expand_a_pair((stack1, dict1), (stack2, dict2), expand_first = use_first, careful_bookkeeping = False)
        if result == 1:
            return generate_path((stack1,dict1),(stack2,dict2),use_first)
        elif result == -1:
            return False
        
        use_first = not use_first

In [172]:
class PageNode:
    def __init__(self,page):
        self.page = page
        
    def __str__(self):
        return str(self.page)
    
    def __repr__(self):
        return repr(self.page)
    
    def __eq__(self, other):
        try:
            return self.page == other.page
        except:
            # If other does not have a page attribute
            # (Main intended use case: If other == None)
            return False
    
    def __hash__(self):
        return hash(self.page)
        
    def get_successors(self):
        return [PageNode(page) for page in list(self.page.linkedPages())]
    
    def get_predecessors(self):
        return [PageNode(page) for page in list(self.page.backlinks())]

In [18]:
site = pywikibot.Site("en", "wikipedia")

In [181]:
start = PageNode(pywikibot.Page(site,"Moors"))
end = PageNode(pywikibot.Page(site,"Aryan"))
bidirectional_DFS(start,end)

[Page('Moors'),
 Category('Category:Wikipedia articles with SUDOC identifiers'),
 Page('Module:Authority control'),
 Page('Module talk:Authority control/testcases'),
 Page('Module talk:Authority control/sandbox/testcases'),
 Category('Category:Redirects from moves'),
 Page('Template:Redirect category shell'),
 Category('Category:Unprintworthy redirects'),
 Category('Category:Templates for unprintworthy redirects'),
 Category('Category:Wikipedia administration'),
 Page('Wikipedia:Bots/Requests for approval'),
 User('User:Ahechtbot'),
 User('User:Ahechtbot/Task 2 list'),
 Page('User talk:5 albert square/Archive 14'),
 Page('User talk:5 albert square/Archive 20'),
 Page('User talk:5 albert square/Archive 19'),
 Page('User talk:5 albert square/Archive 18'),
 Page('User talk:5 albert square/Archive 17'),
 Page('User talk:Davey2010/Archive 2014'),
 Page('User talk:Davey2010/Archive 2014 2'),
 Page('User talk:Davey2010/Archive 2017'),
 Page('User talk:Davey2010/Archive 2017 1'),
 Page('User t